<a href="https://colab.research.google.com/github/hrbolek/learning/blob/master/notebooks/ais/06_sqla.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [30]:
!pip install sqlalchemy
!pip install aiosqlite

In [31]:
from uuid import uuid4, UUID
from sqlalchemy import Column, Uuid
uuid = uuid4

def UUIDColumn():
    return Column(Uuid, primary_key=True, comment="primary key", default=uuid)

def UUIDFKey(comment=None, nullable=True, **kwargs):
    return Column(Uuid, index=True, comment=comment, nullable=nullable, **kwargs)

In [32]:
from sqlalchemy.orm import declarative_base
BaseModel = declarative_base()

In [33]:
import sqlalchemy
from sqlalchemy import (
    Column,
    String,
    DateTime,
    Boolean,
)
from sqlalchemy.orm import relationship
from sqlalchemy.ext.hybrid import hybrid_property

class UserModel(BaseModel):
    """Spravuje data spojena s uzivatelem"""

    __tablename__ = "users"

    id = UUIDColumn()
    name = Column(String)
    surname = Column(String)

    @hybrid_property
    def fullname(self):
        return self.name + " " + self.surname

    email = Column(String)
    startdate = Column(DateTime, comment="first date of user in the system")
    enddate = Column(DateTime, comment="last date of user in the system")
    valid = Column(Boolean, default=True, comment="if the user is still active")

    memberships = relationship("MembershipModel", back_populates="user", foreign_keys="MembershipModel.user_id")
    roles = relationship("RoleModel", back_populates="user", foreign_keys="RoleModel.user_id")
    # groups = relationship("GroupModel",
    #     secondary="join(MembershipModel, GroupModel, GroupModel.id==MembershipModel.group_id)",
    #     primaryjoin="UserModel.id==MembershipModel.user_id",
    #     secondaryjoin="GroupModel.id==MembershipModel.group_id",
    #     uselist=True,
    #     viewonly=True
    # )

    created = Column(DateTime, server_default=sqlalchemy.sql.func.now(), comment="when record has been created")
    lastchange = Column(DateTime, server_default=sqlalchemy.sql.func.now(), comment="timestamp")
    createdby = UUIDFKey(nullable=True, comment="who has created this record")#Column(ForeignKey("users.id"), index=True, nullable=True)
    changedby = UUIDFKey(nullable=True, comment="who has changed this record")#Column(ForeignKey("users.id"), index=True, nullable=True)

    rbacobject = UUIDFKey(nullable=True, comment="holds object for role resolution")#Column(ForeignKey("users.id"), index=True, nullable=True)

In [34]:
from sqlalchemy import (
    ForeignKey
)
class GroupModel(BaseModel):
    """Spravuje data spojena se skupinou"""

    __tablename__ = "groups"

    id = UUIDColumn()
    name = Column(String, comment="name of the group")
    name_en = Column(String, comment="english name of the group")
    email = Column(String, comment="can be an email for whole group")

    startdate = Column(DateTime, comment="born date of the group")
    enddate = Column(DateTime, comment="date when group `died`")
    valid = Column(Boolean, default=True, comment="if the group still exists")

    mastergroup_id = Column(ForeignKey("groups.id"), index=True, comment="link to the commanding group")

    memberships = relationship("MembershipModel", back_populates="group")
    roles = relationship("RoleModel", back_populates="group")

    created = Column(DateTime, server_default=sqlalchemy.sql.func.now(), comment="when record has been created")
    lastchange = Column(DateTime, server_default=sqlalchemy.sql.func.now(), comment="timestamp")
    createdby = UUIDFKey(nullable=True, comment="who has created this record")#Column(ForeignKey("users.id"), index=True, nullable=True)
    changedby = UUIDFKey(nullable=True, comment="who has changed this record")#Column(ForeignKey("users.id"), index=True, nullable=True)

    rbacobject = UUIDFKey(nullable=True, comment="holds object for role resolution")#Column(ForeignKey("users.id"), index=True, nullable=True)

In [35]:
class MembershipModel(BaseModel):
    """Spojuje User s Group jestlize User je clen Group
    Umoznuje udrzovat historii spojeni
    """

    __tablename__ = "memberships"

    id = UUIDColumn()
    user_id = Column(ForeignKey("users.id"), index=True)
    group_id = Column(ForeignKey("groups.id"), index=True)

    startdate = Column(DateTime, comment="first date of membership")
    enddate = Column(DateTime, comment="last date of membership")
    valid = Column(Boolean, default=True, comment="if the membership is still active")

    created = Column(DateTime, server_default=sqlalchemy.sql.func.now(), comment="when record has been created")
    lastchange = Column(DateTime, server_default=sqlalchemy.sql.func.now(), comment="timestamp")
    createdby = UUIDFKey(nullable=True, comment="who has created this record")#Column(ForeignKey("users.id"), index=True, nullable=True)
    changedby = UUIDFKey(nullable=True, comment="who has changed this record")#Column(ForeignKey("users.id"), index=True, nullable=True)

    user = relationship("UserModel", back_populates="memberships", foreign_keys=[user_id])
    group = relationship("GroupModel", back_populates="memberships")

    rbacobject = UUIDFKey(nullable=True, comment="holds object for role resolution")#Column(ForeignKey("users.id"), index=True, nullable=True)

In [36]:
class RoleModel(BaseModel):
    """Spojuje uzivatele a skupinu, ve ktere uzivatel "hraje" roli"""

    __tablename__ = "roles"

    id = UUIDColumn()
    user_id = Column(ForeignKey("users.id"), index=True)
    group_id = Column(ForeignKey("groups.id"), index=True)
    roletype_id = Column(ForeignKey("roletypes.id"), index=True)

    startdate = Column(DateTime, comment="when the role begins")
    enddate = Column(DateTime, comment="when the role ends")
    valid = Column(Boolean, default=True, comment="if the role is still active")

    created = Column(DateTime, server_default=sqlalchemy.sql.func.now(), comment="when record has been created")
    lastchange = Column(DateTime, server_default=sqlalchemy.sql.func.now(), comment="timestamp")
    createdby = UUIDFKey(nullable=True, comment="who has created this record")#Column(ForeignKey("users.id"), index=True, nullable=True)
    changedby = UUIDFKey(nullable=True, comment="who has changed this record")#Column(ForeignKey("users.id"), index=True, nullable=True)

    roletype = relationship("RoleTypeModel", back_populates="roles")
    user = relationship("UserModel", back_populates="roles", foreign_keys=[user_id])
    group = relationship("GroupModel", back_populates="roles")

    rbacobject = UUIDFKey(nullable=True, comment="holds object for role resolution")#Column(ForeignKey("users.id"), index=True, nullable=True)

In [37]:
class RoleTypeModel(BaseModel):
    """Urcuje typ role (Vedouci katedry, dekan apod.)"""

    __tablename__ = "roletypes"

    id = UUIDColumn()
    name = Column(String, comment="name of the type")
    name_en = Column(String, comment="english name of the type")

    roles = relationship("RoleModel", back_populates="roletype")

    created = Column(DateTime, server_default=sqlalchemy.sql.func.now(), comment="when record has been created")
    lastchange = Column(DateTime, server_default=sqlalchemy.sql.func.now(), comment="timestamp")
    createdby = UUIDFKey(nullable=True, comment="who has created this record")#Column(ForeignKey("users.id"), index=True, nullable=True)
    changedby = UUIDFKey(nullable=True, comment="who has changed this record")#Column(ForeignKey("users.id"), index=True, nullable=True)

    rbacobject = UUIDFKey(nullable=True, comment="holds object for role resolution")#Column(ForeignKey("users.id"), index=True, nullable=True)

In [38]:
from sqlalchemy.ext.asyncio import AsyncSession
from sqlalchemy.ext.asyncio import create_async_engine
from sqlalchemy.orm import sessionmaker

async def startEngine(connectionstring="sqlite+aiosqlite://", makeDrop=False, makeUp=True) -> AsyncSession:
    global dbInitIsDone
    """Provede nezbytne ukony a vrati asynchronni SessionMaker"""

    asyncEngine = create_async_engine(connectionstring, echo=True)

    async with asyncEngine.begin() as conn:
        if makeDrop:
            await conn.run_sync(BaseModel.metadata.drop_all)
            print("BaseModel.metadata.drop_all finished")
        if makeUp:
            try:
                await conn.run_sync(BaseModel.metadata.create_all)
                print("BaseModel.metadata.create_all finished")
            except sqlalchemy.exc.NoReferencedTableError as e:
                print(e)
                print("Unable automaticaly create tables")
                return None
    dbInitIsDone = True

    async_sessionMaker = sessionmaker(
        asyncEngine, expire_on_commit=False, class_=AsyncSession
    )
    return async_sessionMaker



In [39]:
async_session_maker = await startEngine()


2024-03-21 18:53:28,929 INFO sqlalchemy.engine.Engine BEGIN (implicit)


INFO:sqlalchemy.engine.Engine:BEGIN (implicit)


2024-03-21 18:53:28,936 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("users")


INFO:sqlalchemy.engine.Engine:PRAGMA main.table_info("users")


2024-03-21 18:53:28,942 INFO sqlalchemy.engine.Engine [raw sql] ()


...

INFO:sqlalchemy.engine.Engine:CREATE INDEX ix_roles_createdby ON roles (createdby)


2024-03-21 18:53:29,173 INFO sqlalchemy.engine.Engine [no key 0.00218s] ()


INFO:sqlalchemy.engine.Engine:[no key 0.00218s] ()


BaseModel.metadata.create_all finished
2024-03-21 18:53:29,176 INFO sqlalchemy.engine.Engine COMMIT


INFO:sqlalchemy.engine.Engine:COMMIT


In [45]:
async def AddUser(**attrs):
    async with async_session_maker.begin() as session:
        u = UserModel(**attrs)
        session.add(u)
        await session.commit()
        return u

u = await AddUser(name="John", surname="Newbie", id=UUID("8b22e698f41548ab954a4554568839c5"))
print(u)

2024-03-21 19:02:10,323 INFO sqlalchemy.engine.Engine BEGIN (implicit)


INFO:sqlalchemy.engine.Engine:BEGIN (implicit)


2024-03-21 19:02:10,336 INFO sqlalchemy.engine.Engine INSERT INTO users (id, name, surname, email, startdate, enddate, valid, createdby, changedby, rbacobject) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?) RETURNING created, lastchange


INFO:sqlalchemy.engine.Engine:INSERT INTO users (id, name, surname, email, startdate, enddate, valid, createdby, changedby, rbacobject) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?) RETURNING created, lastchange


2024-03-21 19:02:10,340 INFO sqlalchemy.engine.Engine [generated in 0.00489s] ('8b22e698f41548ab954a4554568839c5', 'John', 'Newbie', None, None, None, 1, None, None, None)


INFO:sqlalchemy.engine.Engine:[generated in 0.00489s] ('8b22e698f41548ab954a4554568839c5', 'John', 'Newbie', None, None, None, 1, None, None, None)


2024-03-21 19:02:10,350 INFO sqlalchemy.engine.Engine COMMIT


INFO:sqlalchemy.engine.Engine:COMMIT


In [46]:
from sqlalchemy import select
async def GetUser(id):
    stmt = select(UserModel).filter_by(id=id)
    async with async_session_maker.begin() as session:
        results = await session.execute(stmt)
        return results.scalar()

u = await GetUser(id=UUID("8b22e698f41548ab954a4554568839c5"))
print(u)

2024-03-21 19:02:35,400 INFO sqlalchemy.engine.Engine BEGIN (implicit)


INFO:sqlalchemy.engine.Engine:BEGIN (implicit)


2024-03-21 19:02:35,407 INFO sqlalchemy.engine.Engine SELECT users.id, users.name, users.surname, users.email, users.startdate, users.enddate, users.valid, users.created, users.lastchange, users.createdby, users.changedby, users.rbacobject 
FROM users 
WHERE users.id = ?


INFO:sqlalchemy.engine.Engine:SELECT users.id, users.name, users.surname, users.email, users.startdate, users.enddate, users.valid, users.created, users.lastchange, users.createdby, users.changedby, users.rbacobject 
FROM users 
WHERE users.id = ?


2024-03-21 19:02:35,414 INFO sqlalchemy.engine.Engine [cached since 226s ago] ('8b22e698f41548ab954a4554568839c5',)


INFO:sqlalchemy.engine.Engine:[cached since 226s ago] ('8b22e698f41548ab954a4554568839c5',)


2024-03-21 19:02:35,419 INFO sqlalchemy.engine.Engine COMMIT


INFO:sqlalchemy.engine.Engine:COMMIT


In [47]:
from sqlalchemy import delete
async def DeleteUser(id):
    stmt = delete(UserModel).filter_by(id=id)
    async with async_session_maker.begin() as session:
        results = await session.execute(stmt)
        return results

r = await DeleteUser(id=UUID("8b22e698f41548ab954a4554568839c5"))
print(r)

2024-03-21 19:02:41,564 INFO sqlalchemy.engine.Engine BEGIN (implicit)


INFO:sqlalchemy.engine.Engine:BEGIN (implicit)


2024-03-21 19:02:41,568 INFO sqlalchemy.engine.Engine DELETE FROM users WHERE users.id = ?


INFO:sqlalchemy.engine.Engine:DELETE FROM users WHERE users.id = ?


2024-03-21 19:02:41,571 INFO sqlalchemy.engine.Engine [cached since 74.11s ago] ('8b22e698f41548ab954a4554568839c5',)


INFO:sqlalchemy.engine.Engine:[cached since 74.11s ago] ('8b22e698f41548ab954a4554568839c5',)


2024-03-21 19:02:41,575 INFO sqlalchemy.engine.Engine COMMIT


INFO:sqlalchemy.engine.Engine:COMMIT
